In [1]:
import os, sys, errno, glob
import tensorflow as tf
import numpy as np
import pandas as pd
import cellCNN_utils  
from cellCNN_utils import loadFCS, ftrans, mkdir_p, get_items, generate_data, generate_normalized_data
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
d = Path().resolve()
sys.path.append(d)
%pylab inline
# define input and output directories
FCS_DATA_PATH = os.path.join(d, 'FlowRepository')
# select the relevant markers for further analysis
data_fcs = loadFCS(glob.glob(FCS_DATA_PATH + '/discovery_cohort.fcs')[0], transform=None, auto_comp=False)
print(data_fcs.channels)

/opt/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Populating the interactive namespace from numpy and matplotlib
['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3', 'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10', 'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a', 'gate_source', 'manual_labels', 'labels', 'cell_id', 'cd4_labels', 'cd8_labels', 'run_0', 'run_1', 'run_2', 'intersection_3_runs']


In [2]:
#-------------------------------SET PARAMS HERE -----------------------#

cg = 'RRMS' #Control group, set to NIND or RRMS depending on experimental setting
ncells = 100 #num cells per multi-cell input or multi-cell test data
ncell_test = 5000 #num cells for phenotype prediction
ntrain_all = 30000 # number of multi-cell inputs for the train data (in total)
ntest_all = 10000 # number of multi-cell inputs for the test data
nfilter = 8 #number of filters for the training

#-----------------------------------------------------------------------#

markers=['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 
        'CD25', 'CD27', 'CD28', 'CD3', 'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 
        'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10', 'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3',
        'IL.4', 'IL.6', 'IL.9', 'TNF.a']
len(markers)

35

In [3]:
#gate_source is 35.index
#gate_source is ind=1 in excell, label is ind=4
#match gate_source from table NINDC=0, RRMS=1
metadata= pd.read_excel(FCS_DATA_PATH+'/meta_data_discovery_cohort.xlsx')
metadata=metadata.to_numpy()
gate_source = metadata[:,1]
labelsTemp = metadata[:,4]
data = []
sample_labels =[]
for i in range(99):
    cur_gs = gate_source[i]
    cur_lab = labelsTemp[i]
    patient_sample = []
    if cur_lab == 'HD':
        gs_ind = np.where(data_fcs.events[:,35]==cur_gs)
        for j in gs_ind[0]:
            patient_sample.append(data_fcs.events[j,0:35])
        sample_labels.append(0)
    elif cur_lab == cg:
        gs_ind = np.where(data_fcs.events[:,35]==cur_gs)
        for j in gs_ind[0]:
            patient_sample.append(data_fcs.events[j,0:35])
        sample_labels.append(1)
    if len(patient_sample)>0:
        data.append(np.asarray(patient_sample))
sample_labels=np.asarray(sample_labels)


In [4]:
# Here we randomly split the samples in training/test sets.

def train_test_split(train_idx1=[], train_idx2=[], test=True):
    # set random seed for reproducible results and monte-carlo repetitions only for training!
#     np.random.seed(4)
    
    # split the fcs files into training, validation and test set (note that secure-protocols do not use validation sets)
    group1 = np.where(sample_labels == 0)[0]
    group2 = np.where(sample_labels == 1)[0]
    l1, l2 = len(group1), len(group2)
    ntrain_per_class = 24
    ntest_group1 = l1 - ntrain_per_class
    ntest_group2 = l2 - ntrain_per_class

    # get the sample indices
    train_idx1 = list(np.random.choice(group1, size=ntrain_per_class, replace=False))
    test_idx1 = [i for i in group1 if i not in train_idx1]
    train_idx2 = list(np.random.choice(group2, size=ntrain_per_class, replace=False))
    test_idx2 = [i for i in group2 if i not in train_idx2]

    print("test indices")
    test_indices = [test_idx1,test_idx2]
    print(test_indices) 
    print("train indices")
    print(train_idx1)
    print(train_idx2)

    train_indices = [train_idx1,train_idx2]

    # load the training samples
    group1_list, group2_list = [], []
    for idx in train_idx1:
        x = data[idx][:]
        group1_list.append(x)

    for idx in train_idx2:
        x = data[idx][:]
        group2_list.append(x)

    # load the test samples
    t_group1_list, t_group2_list = [], []
    test_phenotypes = []
    for idx in test_idx1:
        x = data[idx][:]
        t_group1_list.append(x)
        test_phenotypes.append(0)

    for idx in test_idx2:
        x = data[idx][:]
        t_group2_list.append(x)
        test_phenotypes.append(1)

    # finally prepare training data
    cut = int(1 * len(group1_list))
    train_samples = group1_list[:cut] + group2_list[:cut]
    train_phenotypes = [0] * len(group1_list[:cut]) + [1] * len(group2_list[:cut])
    valid_samples = group1_list[cut:] + group2_list[cut:]
    valid_phenotypes = [0] * len(group1_list[cut:]) + [1] * len(group2_list[cut:])
    test_samples = t_group1_list + t_group2_list
    print(test_phenotypes)
    return train_samples,train_phenotypes,test_samples,test_phenotypes, test_indices,train_indices



In [5]:

#debug, for changes introduced to cellCNN_utils
# import importlib
# importlib.reload(cellCNN_utils)

train_samples, train_phenotypes, test_samples, test_phenotypes, test_indices, train_indices = train_test_split()
# generate ntrain_all (ntrain_all/2 per phenotype) training samples for centralized test!

# scaler,x_tr,y_tr = generate_data(train_samples, train_phenotypes, 'Flow/', generate_valid_set=False, 
#                                                ncell=ncells, nsubset=int(ntrain_all/2), scale=True, 
#                                                per_sample=False, verbose=0, saveFile=False,subset_selection = 'random')


scaler,x_test,y_test = generate_data(test_samples, test_phenotypes, 'Flow/', generate_valid_set=False, 
                                               ncell=ncells, nsubset=5000, scale=True, 
                                               per_sample=False, verbose=0, saveFile=True,
                                               subset_selection = 'random', generateAsTest=True)

test indices
[[19, 44, 50, 53, 55], [6, 7, 11, 13, 25, 37, 41]]
train indices
[57, 58, 43, 45, 0, 10, 49, 26, 21, 17, 15, 54, 36, 18, 35, 48, 20, 30, 27, 31, 52, 51, 56, 1]
[8, 9, 2, 4, 40, 28, 5, 34, 39, 22, 47, 42, 16, 23, 59, 33, 12, 29, 32, 14, 24, 3, 38, 46]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
new scale
Generating multi-cell inputs...
Done.


In [6]:
from sklearn.utils import shuffle

#generate also the test set on full min-ncell per sample:
def generate_for_pheno_prediction(new_samples,phenotypes,scaler):
        ncell_per_sample = np.min([x.shape[0] for x in new_samples])
        print(f"Predictions based on multi-cell inputs containing {ncell_per_sample} cells.")
        nmark = len(new_samples[0][1])
        # scale the new samples if we did that for the training samples
        if scaler is not None:
            new_samples = [scaler.transform(x) for x in new_samples]
        new_samples = [shuffle(x)[:ncell_per_sample].reshape(1, ncell_per_sample, nmark)
                           for x in new_samples]
        data_test = np.vstack(new_samples)
        mkdir_p('Flow/X_test_all/')
        for i in range(len(data_test)):
            np.savetxt('Flow/' + 'X_test_all/' + str(i) +'.txt', (transpose(data_test[i])))
        np.savetxt('Flow/' + 'y_test_all.txt', phenotypes)
        return data_test,phenotypes,ncell_per_sample
    
data_test,phenotypes,ncell_per_sample = generate_for_pheno_prediction(test_samples,test_phenotypes,scaler)
print(shape(data_test))
print(phenotypes)

Predictions based on multi-cell inputs containing 10173 cells.
(12, 10173, 35)
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# import importlib
# importlib.reload(cellCNN_utils)

# Here we randomly split the samples for n hosts
nhosts=6

test_idx1 = test_indices[0]
test_idx2 = test_indices[1]

train_idx1 = train_indices[0]
train_idx2 = train_indices[1]

train_idx1 = shuffle(train_idx1)
train_idx2 = shuffle(train_idx2)

print("Test set indices:")
print(test_idx1)
print(test_idx2)
print("Global train set indices:")

#to take the runs on 10 different distributions (for box plot)

train_idx1 = shuffle(train_idx1)
train_idx2 = shuffle(train_idx2)
print(train_idx1)
print(train_idx2)

#distribute train indices balanced among n hosts:
split_idx_1 = []
split_idx_2 = []
group1_list = np.flip(np.array_split(numpy.array(train_idx1), nhosts))
group2_list = numpy.array_split(numpy.array(train_idx2), nhosts)

for i in range(nhosts):
    split_idx_1.append(group1_list[i].tolist())
    split_idx_2.append(group2_list[i].tolist())

print("Global train splitted among hosts - indices:")
print(split_idx_1)
print(split_idx_2)

xtr = []
ytr=[]
for i in range(nhosts):
    print("\nHost no.", i, ":")
    folder_path = 'splitFlow' + str(nhosts) + '/host' + str(i) + '/'
    host_idx_1 = split_idx_1[i]
    host_idx_2 = split_idx_2[i]
    print("host_idx_1:", host_idx_1, "- host_idx_2:", host_idx_2)
     # load the training samples
    host_group1_list, host_group2_list = [], []
    train_samples,train_phenotypes = [],[]
    for idx in host_idx_1:
        x = data[idx][:]
        host_group1_list.append(x)

    for idx in host_idx_2:
        x = data[idx][:]
        host_group2_list.append(x)

    # finally prepare training and vallidation data
    cut = int(1 * len(host_group1_list))
    train_samples = host_group1_list[:cut] + host_group2_list[:cut]
    train_phenotypes = [0] * len(host_group1_list[:cut]) + [1] * len(host_group2_list[:cut])
    print(train_phenotypes)
    scaler,x_tr,y_tr = generate_data(train_samples, train_phenotypes, folder_path, scale=True, ncell=ncells, 
                  nsubset=int(ntrain_all/(2*nhosts)),per_sample=False, verbose=0,generate_valid_set=False,
                                     saveFile=True,scaler=scaler,subset_selection = 'random',oneFile=None)
    xtr.append(x_tr)
    ytr.append(y_tr)
x_tr=np.reshape(xtr, (ntrain_all,35,ncells))
y_tr=np.reshape(ytr, (ntrain_all,1))
print(len(x_tr))
print(len(y_tr))

Test set indices:
[19, 44, 50, 53, 55]
[6, 7, 11, 13, 25, 37, 41]
Global train set indices:
[54, 27, 20, 15, 48, 52, 18, 31, 45, 17, 30, 49, 21, 51, 1, 10, 43, 57, 26, 56, 35, 0, 36, 58]
[28, 40, 8, 38, 5, 29, 42, 39, 47, 23, 24, 4, 14, 22, 46, 16, 12, 33, 32, 59, 34, 9, 2, 3]
Global train splitted among hosts - indices:
[[58, 36, 0, 35], [56, 26, 57, 43], [10, 1, 51, 21], [49, 30, 17, 45], [31, 18, 52, 48], [15, 20, 27, 54]]
[[28, 40, 8, 38], [5, 29, 42, 39], [47, 23, 24, 4], [14, 22, 46, 16], [12, 33, 32, 59], [34, 9, 2, 3]]

Host no. 0 :
host_idx_1: [58, 36, 0, 35] - host_idx_2: [28, 40, 8, 38]
[0, 0, 0, 0, 1, 1, 1, 1]
old scale
Generating multi-cell inputs...
Done.

Host no. 1 :
host_idx_1: [56, 26, 57, 43] - host_idx_2: [5, 29, 42, 39]
[0, 0, 0, 0, 1, 1, 1, 1]
old scale
Generating multi-cell inputs...
Done.

Host no. 2 :
host_idx_1: [10, 1, 51, 21] - host_idx_2: [47, 23, 24, 4]
[0, 0, 0, 0, 1, 1, 1, 1]
old scale
Generating multi-cell inputs...
Done.

Host no. 3 :
host_idx_1: [49, 

In [ ]:
#Simulating distribution and encryption to find best
#learning rate and momentum to be tried on actual implementation

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from tensorflow.keras import layers, initializers, regularizers, optimizers, callbacks
from keras import backend as K

def model_pred(prob):
    pred = []
    for p in prob:
        if p[0]>p[1]:
            pred.append(0)
        else:
            pred.append(1)
    return pred
def sigmoidApprox(x):
    degree = 3
    interval = 3
    if degree == 3:  
        if interval == 3:
            return 0.5 + 0.6997*K.pow(x/interval,1)-0.2649*K.pow(x/interval,3)
        if interval == 5:
            return 0.5 + 0.9917*K.pow(x/interval,1)-0.5592*K.pow(x/interval,3)
        if interval == 7:
            return 0.5 + 1.1511*K.pow(x/interval,1)-0.7517*K.pow(x/interval,3)
        if interval == 8:
            return 0.5 + 1.2010*K.pow(x/interval,1)-0.8156*K.pow(x/interval,2)
        if interval == 12:
            return 0.5 + 1.2384*K.pow(x/interval,1)-0.8647*K.pow(x/interval,2)
#repeat cellcnn original training on full train data

y_tr_n = to_categorical(y_tr)

def pool_top_k(x, k):
    return tf.reduce_mean(tf.sort(x, axis=1, direction='DESCENDING')[:, :k, :], axis=1)
def create_model (k,ncell,nfilter,lr,m):
        
        data_input = keras.Input(shape=(ncell, 35))
        coeff_l1=0
        coeff_l2=1e-4
        n_classes=2
        conv = layers.Conv1D(filters=nfilter,
                             kernel_size=1,
                             activation='linear',
                             kernel_initializer=initializers.GlorotNormal(),
                             name='conv1')(data_input)

        # the cell grouping part (top-k pooling)
        pooled = layers.Lambda(pool_top_k, output_shape=(nfilter,), arguments={'k': k})(conv)
        output = layers.Dense(units=n_classes,
                                  activation=sigmoidApprox,
                                  kernel_initializer=initializers.RandomUniform(),
                                  name='output')(pooled)
        model = keras.Model(inputs=data_input, outputs=output)

        model.compile(optimizer=optimizers.SGD(learning_rate=lr,momentum=m),
                          loss='mean_squared_error',
                          metrics=['accuracy'])
        return model
lrs=[0.01,0.02,0.001, 0.005, 0.0001,0.0005]
ms=[0.9,0.7]

for l in lrs:
    for m in ms:
        model = create_model(ncells,ncells,nfilter,l,m)
        #generate data
        x_tr =np.asarray(x_tr)
        x_tr_n = x_tr.transpose(0,2,1)
        history = model.fit(x_tr_n, y_tr_n,
                    batch_size=8,
                    epochs=30,
                    verbose=2,
                    validation_split=0)
        #select top ntest_all number of test multi-cells
        
        x_test = np.asarray(x_test)
        x_test_n = x_test.transpose(0,2,1)
        x_test_n = x_test_n[0:ntest_all,:]

        y_test_n = to_categorical(y_test)
        y_test_n = y_test_n[0:ntest_all,:]
        y_test = y_test[0:ntest_all]
        
        print("learning rate is,",l)
        print("momentum is,",m)
        loss, accuracy = model.evaluate(x_test_n, y_test_n, verbose=0)
        #score = model.evaluate(x_test_n, y_test_n, verbose=0)
        print(f'Test loss: {loss}, Test accuracy: {accuracy}')

        y_pred = model.predict(x_test_n)
        y_pred = model_pred(y_pred)


        from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("F-score:",f1_score(y_test, y_pred))
        print("precision:",precision_score(y_test, y_pred))
        print("recall:",recall_score(y_test, y_pred)) 
        #For phenotype predictions on test set using all cells 
        print(ncell_test)
        print(len(data_test[0]))
        model2 = create_model(11663, 11663, nfilter,l,m)
        weights = model.get_weights()
        model2.set_weights(weights)
        
        data_test = np.array(data_test)
        print(len(data_test))
        data_test_n = data_test
#         .transpose(0,2,1)
        phenotypes_n = to_categorical(phenotypes)

        y_pred = model2.predict(data_test_n)
        y_pred = model_pred(y_pred)
        # print(y_pred)
        from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
        print("Accuracy:", accuracy_score(phenotypes, y_pred))
        print("F-score:",f1_score(phenotypes, y_pred))
        print("precision:",precision_score(phenotypes, y_pred))
        print("recall:",recall_score(phenotypes, y_pred)) 

In [ ]:
#The reproduction of original CellCnn model training without validation test and further analysis part
#This part is used for the comparison of accuracy/precision/recall/f-score of CellCnn with secure distributed version
#test on 100-cell multi-instances and full test set phenotype prediction
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from tensorflow.keras import layers, initializers, regularizers, optimizers, callbacks
from keras import backend as K

#repeat cellcnn original training on full train data

y_tr_n = to_categorical(y_tr)

def pool_top_k(x, k):
    return tf.reduce_mean(tf.sort(x, axis=1, direction='DESCENDING')[:, :k, :], axis=1)
def create_model (k,ncell,nfilter):
        
        data_input = keras.Input(shape=(ncell, 35))
        coeff_l1=0
        coeff_l2=1e-4
        n_classes=2
        # the filters
        conv = layers.Conv1D(filters=nfilter,
                             kernel_size=1,
                             activation='relu',
                             kernel_initializer=initializers.RandomUniform(),
                             kernel_regularizer=regularizers.l1_l2(l1=coeff_l1, l2=coeff_l2),
                             name='conv1')(data_input)

        # the cell grouping part (top-k pooling)
        pooled = layers.Lambda(pool_top_k, output_shape=(nfilter,), arguments={'k': k})(conv)
        output = layers.Dense(units=n_classes,
                                  activation='softmax',
                                  kernel_initializer=initializers.RandomUniform(),
                                  kernel_regularizer=regularizers.l1_l2(l1=coeff_l1, l2=coeff_l2),
                                  name='output')(pooled)
        model = keras.Model(inputs=data_input, outputs=output)

        model.compile(optimizer=optimizers.Adam(learning_rate=0.01),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])
        return model
model = create_model(ncells,ncells,nfilter)
#generate data

x_tr_n = x_tr.transpose(0,2,1)
# Fit data to model
shuffler = np.random.permutation(len(x_tr_n))
x_tr_n = x_tr_n[shuffler]
y_tr_n = y_tr_n[shuffler]

history = model.fit(x_tr_n, y_tr_n,
            batch_size=64,
            epochs=30,
            verbose=2,
            validation_split=0)

# list all data in history
# print(history.history.keys())
# # summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
accmulti,precmulti,recallmulti,acccel,preccel,recallcel = 0,0,0,0,0,0

In [ ]:
#For 100-cell predictions on test set
def model_pred(prob):
    pred = []
    for p in prob:
        if p[0]>p[1]:
            pred.append(0)
        else:
            pred.append(1)
    return pred

x_test_n = x_test.transpose(0,2,1)
x_test_n = x_test_n[0:ntest_all,:]

y_test_n = to_categorical(y_test)
y_test_n = y_test_n[0:ntest_all,:]
loss, accuracy = model.evaluate(x_test_n, y_test_n, verbose=0)
#score = model.evaluate(x_test_n, y_test_n, verbose=0)
print("For 100-cell predictions on test set with size",x_test_n.shape)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')

y_pred = model.predict(x_test_n)
y_pred = model_pred(y_pred)
y_test= y_test[0:ntest_all]

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
accmulti += accuracy_score(y_test, y_pred)
precmulti += precision_score(y_test, y_pred)
recallmulti += recall_score(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F-score:",f1_score(y_test, y_pred))
print("precision:",precision_score(y_test, y_pred))
print("recall:",recall_score(y_test, y_pred)) 
#For phenotype predictions on test set using all cells 

model2 = create_model(10173, 10173,nfilter)
weights = model.get_weights()
model2.set_weights(weights)
data_test_n = data_test
# .transpose(0,2,1)
phenotypes_n = to_categorical(phenotypes)

y_pred = model2.predict(data_test_n)
y_pred = model_pred(y_pred)
# print(y_pred)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
print("Accuracy:", accuracy_score(phenotypes, y_pred))
print("F-score:",f1_score(phenotypes, y_pred))
print("precision:",precision_score(phenotypes, y_pred))
print("recall:",recall_score(phenotypes, y_pred)) 
acccel += accuracy_score(phenotypes, y_pred)
preccel += precision_score(phenotypes, y_pred)
recallcel += recall_score(phenotypes, y_pred)